In [62]:
# !git clone https://github.com/anantSinghCross/xray_classification_pneumonia.git

In [63]:
import glob
base_path = './xray_classification_pneumonia/Dataset_compressed/train'
file_path = list(glob.glob(base_path + "/*/*.*"))
pneumonia = list(glob.glob(base_path+"/PNEUMONIA/*.*"))
normal = list(glob.glob(base_path+"/NORMAL/*.*"))

# pneumonia
# normal

In [64]:
len(pneumonia), len(normal)

(3875, 1341)

In [65]:
file_path[:5]

['./xray_classification_pneumonia/Dataset_compressed/train\\NORMAL\\0.jpg',
 './xray_classification_pneumonia/Dataset_compressed/train\\NORMAL\\1.jpg',
 './xray_classification_pneumonia/Dataset_compressed/train\\NORMAL\\10.jpg',
 './xray_classification_pneumonia/Dataset_compressed/train\\NORMAL\\100.jpg',
 './xray_classification_pneumonia/Dataset_compressed/train\\NORMAL\\1000.jpg']

In [66]:
import matplotlib.pyplot as plt
import os
test = os.path.dirname(file_path[0])
print(test)
class_name = os.path.split(os.path.dirname(test))
print(class_name)

./xray_classification_pneumonia/Dataset_compressed/train\NORMAL
('./xray_classification_pneumonia/Dataset_compressed', 'train')


In [67]:
# folder name to label
labels = []
for fp in file_path:
    tmp = os.path.dirname(fp)
    class_name = os.path.split(tmp)
    if class_name[1] == "PNEUMONIA":
        labels.append(1)
    else:
        labels.append(0)

In [68]:
import numpy as np
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2
img_width = 60
img_height = 60
dataset = np.ndarray(shape=(len(file_path), img_height*img_width), dtype=np.float32)
i=0
for _file in file_path:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (img_width, img_height))
    
    dataset[i] = img_resized.flatten()
    i += 1
    if i % 250 == 0:
        print(f"{i} images to array")
print("Done!")

250 images to array
500 images to array
750 images to array
1000 images to array
1250 images to array
1500 images to array
1750 images to array
2000 images to array
2250 images to array
2500 images to array
2750 images to array
3000 images to array
3250 images to array
3500 images to array
3750 images to array
4000 images to array
4250 images to array
4500 images to array
4750 images to array
5000 images to array
Done!


In [69]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [70]:
x_train,x_test,y_train,y_test = train_test_split(dataset, labels, test_size=0.2,random_state=42)

In [71]:
len(x_train),len(x_test),len(y_train),len(y_test)

(4172, 1044, 4172, 1044)

In [72]:
y_train,y_test = map(lambda x: to_categorical(x),[y_train,y_test])

### 모델생성

In [73]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

In [74]:
def build():
    model = Sequential()
    model.add(Dense(100,activation='relu',input_shape=(3600,)))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    return model

model = build()
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 100)               360100    
                                                                 
 dense_28 (Dense)            (None, 50)                5050      
                                                                 
 dense_29 (Dense)            (None, 2)                 102       
                                                                 
Total params: 365,252
Trainable params: 365,252
Non-trainable params: 0
_________________________________________________________________


In [75]:
model.predict(dataset)

163/163 [==============================] - 0s 736us/step


array([[4.0589401e-13, 1.0000000e+00],
       [1.0000000e+00, 1.6675341e-28],
       [1.0000000e+00, 1.7295543e-21],
       ...,
       [9.9996912e-01, 3.0827970e-05],
       [1.0000000e+00, 2.7162708e-30],
       [1.0000000e+00, 6.8399528e-34]], dtype=float32)

## Adam 학습

In [76]:
model_adam = build()
model_adam.compile(optimizer=optimizers.Adam(learning_rate = 0.001),loss='categorical_crossentropy',metrics=['accuracy'])
history = model_adam.fit(x_train,y_train, validation_data=(x_test,y_test), batch_size=64,epochs=20)

Epoch 1/20
66/66 [==============================] - 1s 4ms/step - loss: 41.2651 - accuracy: 0.7963 - val_loss: 2.4131 - val_accuracy: 0.9291
Epoch 2/20
66/66 [==============================] - 0s 2ms/step - loss: 4.8264 - accuracy: 0.8869 - val_loss: 31.7417 - val_accuracy: 0.5421
Epoch 3/20
66/66 [==============================] - 0s 2ms/step - loss: 11.5515 - accuracy: 0.8456 - val_loss: 3.5159 - val_accuracy: 0.9444
Epoch 4/20
66/66 [==============================] - 0s 2ms/step - loss: 2.5151 - accuracy: 0.9425 - val_loss: 3.7618 - val_accuracy: 0.9262
Epoch 5/20
66/66 [==============================] - 0s 2ms/step - loss: 3.1614 - accuracy: 0.9264 - val_loss: 3.9266 - val_accuracy: 0.9176
Epoch 6/20
66/66 [==============================] - 0s 2ms/step - loss: 2.1091 - accuracy: 0.9437 - val_loss: 2.1306 - val_accuracy: 0.9521
Epoch 7/20
66/66 [==============================] - 0s 3ms/step - loss: 2.2650 - accuracy: 0.9377 - val_loss: 7.2030 - val_accuracy: 0.8630
Epoch 8/20
66/66 

### 예측

In [77]:
import matplotlib.pyplot as plt
import numpy as np

prediction = model_adam.predict(x_test)
prediction_class = np.argmax(prediction,axis=1)
true_classes = np.argmax(y_test, axis=1)

33/33 [==============================] - 0s 753us/step
